In [1]:
import numpy as np
import pandas as pd
import math
import itertools
import matplotlib.pyplot as plt
import sys
import torch  
import numpy as np  
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm 

import import_ipynb
from Environments import Square_Crossroads
from Actor_Critic import Actor_Critic

importing Jupyter notebook from Environments.ipynb
Environment reset with param
Environment reset with param
{0: [[0, 5], [5, 10], 0, 0], 1: [[5, 0], [0, 5], 0, 0], 2: [[5, 10], [5, 0], 0, 0], 3: [[10, 5], [5, 0], 0, 0]}
-6.1
{0: [[0.08860349964315856, 4.966599160633527], [5, 10], 0.09469000063047904, 339.34507444983115], 1: [[5, 0], [0, 5], 0.008788172970378573, 359.99998113091215], 2: [[5, 10], [5, 0], 0.04198821047919725, 7.242274145817169], 3: [[10, 5], [5, 0], 0.01798486468425529, 68.08403365717658]}
-4.1
{0: [[0.13694747027149834, 5.109351119429898], [5, 10], 0.15071582941520942, 71.29096235012538], 1: [[5.0659764663592926, 0.0019950259135468698], [0, 5], 0.06600662271056282, 1.7320076538509204], 2: [[5, 10], [5, 0], 0.09041518470155219, 0.7224464677227801], 3: [[10, 5], [5, 0], 0.11769212522324218, 1.8773429631967968]}
-4.1
{0: [[0.3716668591529583, 5.197444613833532], [5, 10], 0.2507063127907415, 20.57182105260155], 1: [[5.0659764663592926, 0.0019950259135468698], [0, 5], 0.165

In [2]:
n_times_steps = 3
seed = 10
dist_cars = 0.1

env = Square_Crossroads(n_times_steps, seed, dist_cars)
env.reset()
print(env.states)
action_space = {0:[0,0], 1:[0,0], 2:[0,0], 3:[0,0]}
# total_reward = 0

state_tensor = torch.zeros([4,2], dtype=torch.float32)


ac = Actor_Critic(state_tensor, 20, 400)

Environment reset with param
Environment reset with param
{0: [[0, 5], [5, 10], 0, 0], 1: [[5, 0], [0, 5], 0, 0], 2: [[5, 10], [5, 0], 0, 0], 3: [[10, 5], [5, 0], 0, 0]}


In [3]:
def huber_loss(advantage, delta = 1.0):
    
    '''
    Huber loss, by give target and prediction
    
    '''
    
    error = abs(advantage)
    
    
    if error.item() > delta:
        loss = 0.5 * delta ** 2 + delta * (error - delta)

    else: 
        loss = 0.5 * error ** 2
#     loss = 0
    return loss

In [4]:
hyperparameters = {"hidden_size_actor" : 200,
                   "hidden_size_critic" : 400,
                   "lr" : 5e-4,
                   "step_size" : 10,
                   "gamma" : 0.5}

max_episodes = 300
num_steps = 1000
GAMMA = 0.99
delta = 1

In [10]:
def train(env, hyperparameters):
    
    model = Actor_Critic(state_tensor, hyperparameters["hidden_size_actor"], hyperparameters["hidden_size_critic"])
    model_optimizer = optim.Adam(model.parameters(), lr = hyperparameters["lr"])
    model_scheduler = optim.lr_scheduler.StepLR(model_optimizer, step_size = hyperparameters["step_size"], gamma = hyperparameters["gamma"])
    
    all_lengths = []
    average_lengths = []
    all_rewards = []

    for episode in range(max_episodes):
        print("Training episode: ", episode)
        
        log_probs = []
        values = []
        rewards = []

        states = [v[0] for v in env.states.values()] 
        state_space = torch.tensor([[states[0][0], states[0][1]], 
                                    [states[1][0], states[1][1]],
                                    [states[2][0], states[2][1]],
                                    [states[3][0], states[3][1]]], dtype=torch.float32)

        for steps in tqdm(range(num_steps)):
#             print(f"Step {steps} of episode {episode}")
            
            value, policy_dist_alpha, policy_dist_beta = model.forward(state_space)
            
            value = torch.sum(value).detach().numpy()
            

            action_1 = np.random.beta(policy_dist_alpha[0].detach().numpy(), policy_dist_beta[0].detach().numpy(), size=2)
            action_2 = np.random.beta(policy_dist_alpha[1].detach().numpy(), policy_dist_beta[1].detach().numpy(), size=2)
            action_3 = np.random.beta(policy_dist_alpha[2].detach().numpy(), policy_dist_beta[2].detach().numpy(), size=2)
            action_4 = np.random.beta(policy_dist_alpha[3].detach().numpy(), policy_dist_beta[3].detach().numpy(), size=2)

            action_space = torch.tensor([action_1,
                                         action_2,
                                         action_3,
                                         action_4],
                                         dtype=torch.float32)

            new_state_space, reward, done = env.step(action_space)
            rewards.append(reward)
            values.append(value)

            
            states = [v[0] for v in env.states.values()] 
            new_state_space = torch.tensor([[states[0][0], states[0][1]], 
                                            [states[1][0], states[1][1]],
                                            [states[2][0], states[2][1]],
                                            [states[3][0], states[3][1]]], dtype=torch.float32)

            state_space = new_state_space
            
            if done or steps == num_steps-1:
                Qval, policy_dist_alpha, policy_dist_beta = model.forward(state_space)
                Qval = torch.sum(Qval).detach().numpy()
                all_rewards.append(np.sum(rewards))
                all_lengths.append(steps)
                average_lengths.append(np.mean(all_lengths[-10:]))
                if episode % 10 == 0:                    
                    sys.stdout.write("episode: {}, reward: {}, total length: {}, average length: {} \n".format(episode, np.sum(rewards), steps, average_lengths[-1]))
                break
        
        # compute Q values
        Qvals = np.zeros_like(values)

        for t in reversed(range(len(rewards))):
            Qval = rewards[t] + GAMMA * Qval
            Qvals[t] = Qval
        
        #update actor critic
        
        values = [torch.from_numpy(item).float() for item in values]
        values = torch.FloatTensor(values)

        Qvals = torch.FloatTensor(Qvals)

        print(Qvals)
        print(values)
        advantage = (Qvals - values).mean()
        print(advantage)
        loss_critic = huber_loss(advantage, delta)
        
        loss_actor = (-(policy_dist_alpha.sum() + policy_dist_beta.sum() + 1e-5)) * advantage   
            
        total_loss = loss_actor + loss_critic
        
        print(total_loss)
        model_optimizer.zero_grad()
        total_loss.backward()
        model_optimizer.step()
        model_scheduler.step()
        
    

In [11]:
train(env, hyperparameters)

Training episode:  0


 96%|█████████████████████████████████████████████████████████████████████████████████████████▌   | 963/1000 [00:01<00:00, 580.33it/s]

episode: 0, reward: -8099.999999999997, total length: 999, average length: 999.0 


 96%|█████████████████████████████████████████████████████████████████████████████████████████▌   | 963/1000 [00:01<00:00, 611.04it/s]


tensor([-809.9651, -809.9647, -809.9644, -809.9641, -809.9637, -809.9633,
        -809.9630, -809.9626, -809.9622, -809.9618, -809.9614, -809.9610,
        -809.9606, -809.9602, -809.9598, -809.9594, -809.9590, -809.9586,
        -809.9582, -809.9578, -809.9573, -809.9569, -809.9565, -809.9560,
        -809.9556, -809.9551, -809.9547, -809.9542, -809.9537, -809.9533,
        -809.9528, -809.9523, -809.9518, -809.9514, -809.9509, -809.9504,
        -809.9499, -809.9494, -809.9489, -809.9484, -809.9478, -809.9473,
        -809.9468, -809.9462, -809.9457, -809.9451, -809.9446, -809.9440,
        -809.9435, -809.9429, -809.9423, -809.9417, -809.9412, -809.9406,
        -809.9399, -809.9393, -809.9387, -809.9381, -809.9375, -809.9368,
        -809.9362, -809.9355, -809.9349, -809.9343, -809.9336, -809.9329,
        -809.9323, -809.9316, -809.9308, -809.9302, -809.9294, -809.9288,
        -809.9280, -809.9273, -809.9266, -809.9258, -809.9251, -809.9243,
        -809.9236, -809.9228, -809.922

...

tensor([-809.9651, -809.9647, -809.9644, -809.9641, -809.9637, -809.9633,
        -809.9630, -809.9626, -809.9622, -809.9618, -809.9614, -809.9610,
        -809.9606, -809.9602, -809.9598, -809.9594, -809.9590, -809.9586,
        -809.9582, -809.9578, -809.9573, -809.9569, -809.9565, -809.9560,
        -809.9556, -809.9551, -809.9547, -809.9542, -809.9537, -809.9533,
        -809.9528, -809.9523, -809.9518, -809.9514, -809.9509, -809.9504,
        -809.9499, -809.9494, -809.9489, -809.9484, -809.9478, -809.9473,
        -809.9468, -809.9462, -809.9457, -809.9451, -809.9446, -809.9440,
        -809.9435, -809.9429, -809.9423, -809.9417, -809.9412, -809.9406,
        -809.9399, -809.9393, -809.9387, -809.9381, -809.9375, -809.9368,
        -809.9362, -809.9355, -809.9349, -809.9343, -809.9336, -809.9329,
        -809.9323, -809.9316, -809.9308, -809.9302, -809.9294, -809.9288,
        -809.9280, -809.9273, -809.9266, -809.9258, -809.9251, -809.9243,
        -809.9236, -809.9228, -809.922

         -33.4490,  -25.6050,  -17.6819,   -9.6786])
tensor([-1.5946, -1.5946, -1.5946, -1.5946, -1.5946, -1.5946, -1.5946, -1.5946,
        -1.5946, -1.5946, -1.5946, -1.5946, -1.5946, -1.5946, -1.5946, -1.5946,
        -1.5946, -1.5946, -1.5946, -1.5946, -1.5946, -1.5946, -1.5946, -1.5946,
        -1.5946, -1.5946, -1.5946, -1.5946, -1.5946, -1.5946, -1.5946, -1.5946,
        -1.5946, -1.5946, -1.5946, -1.5946, -1.5946, -1.5946, -1.5946, -1.5946,
        -1.5946, -1.5946, -1.5946, -1.5946, -1.5946, -1.5946, -1.5946, -1.5946,
        -1.5946, -1.5946, -1.5946, -1.5946, -1.5946, -1.5946, -1.5946, -1.5946,
        -1.5946, -1.5946, -1.5946, -1.5946, -1.5946, -1.5946, -1.5946, -1.5946,
        -1.5946, -1.5946, -1.5946, -1.5946, -1.5946, -1.5946, -1.5946, -1.5946,
        -1.5946, -1.5946, -1.5946, -1.5946, -1.5946, -1.5946, -1.5946, -1.5946,
        -1.5946, -1.5946, -1.5946, -1.5946, -1.5946, -1.5946, -1.5946, -1.5946,
        -1.5946, -1.5946, -1.5946, -1.5946, -1.5946, -1.5946, -1.59

        -1.5946, -1.5946, -1.5946, -1.5946, -1.5946, -1.5946, -1.5946, -1.5946])
tensor(-728.3767)
tensor(786.1541, grad_fn=<AddBackward0>)
Training episode:  7


 66%|█████████████████████████████████████████████████████████████▍                               | 660/1000 [00:01<00:00, 503.43it/s]


KeyboardInterrupt: 